In [1]:
!nvcc --version
!pip install open3d
!pip install trimesh

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
     |████████████████████████████████| 188.4MB 68kB/s 
     |████████████████████████████████| 634kB 10.8MB/s 


In [2]:
import torch
import open3d as o3d
import numpy as np
import trimesh as tm

import plotly.graph_objects as go
import plotly.express as px

In [3]:
# Read .ply file
input_file = "/cloud_bin_0.ply"
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud

# Downsample
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

# Convert open3d format to numpy array
# Here, you have the point cloud in numpy format. 
pointcloudxyzxyz = np.asarray(downpcd.points) 
pointcloudxxyyzz = np.transpose(pointcloudxyzxyz)

# Check
print(pointcloudxxyyzz.size)
print(pointcloudxyzxyz)
print(pointcloudxxyyzz)

15624
[[-0.58080001 -1.42920001  3.48439999]
 [-0.45165853 -1.31180486  3.44590247]
 [-0.45530136 -1.33861644  3.44534249]
 ...
 [-0.69504347 -0.59686955  2.64939132]
 [-0.57799999  0.27600002  2.26799997]
 [ 0.3990417  -0.59674999  2.65129169]]
[[-0.58080001 -0.45165853 -0.45530136 ... -0.69504347 -0.57799999
   0.3990417 ]
 [-1.42920001 -1.31180486 -1.33861644 ... -0.59686955  0.27600002
  -0.59674999]
 [ 3.48439999  3.44590247  3.44534249 ...  2.64939132  2.26799997
   2.65129169]]


In [4]:
# define function to visualize
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()
        

In [5]:
# visualize test.ply
pcshow(pointcloudxxyyzz[0],pointcloudxxyyzz[1],pointcloudxxyyzz[2])


In [6]:
# voxel size and margin (DONT KNOW HOW TO CHOOSE THE NUMBER)
voxelSize = 0.1; #in meters
voxelMargin = 5; #in margin

# Get center (location) (maybe needed)
objModelpts = downpcd.get_center()
print(" location = "+str(objModelpts))

# get range x y z
objModelRangeX = [min(pointcloudxxyyzz[0])-voxelSize*20,max(pointcloudxxyyzz[0])+voxelSize*20]
print("[min x, max x] = "+str(objModelRangeX))
objModelRangeY = [min(pointcloudxxyyzz[1])-voxelSize*20,max(pointcloudxxyyzz[1])+voxelSize*20]
print("[min y, max y] = "+str(objModelRangeY))
objModelRangeZ = [min(pointcloudxxyyzz[2])-voxelSize*20,max(pointcloudxxyyzz[2])+voxelSize*20]
print("[min z, max z] = "+str(objModelRangeZ))

location = [-0.06922717 -0.3381356   2.32930508]
[min x, max x] = [-3.3336842164658664, 3.49399995803833]
[min y, max y] = [-3.43114287512643, 2.686400032043457]
[min z, max z] = [-1.1955789421734058, 5.48628568649292]


In [10]:
# ------------------------------- point cloud to TDF ---------------------------------------

# compute voxel grid
[gridx, gridy, gridz] = np.meshgrid(np.arange(objModelRangeX[0],objModelRangeX[1], voxelSize),
                                    np.arange(objModelRangeY[0],objModelRangeY[1], voxelSize),
                                    np.arange(objModelRangeZ[0],objModelRangeZ[1], voxelSize))

#build kdtree
print("build kdtree")
from scipy.spatial import KDTree
modelKDT = KDTree(pointcloudxyzxyz)
#print(gridx.reshape(-1).shape)
#print(gridy.reshape(-1).shape)
#print(gridz.reshape(-1).shape)

# voxel for KDTree
voxelxxyyzz = np.array([gridx.reshape(-1), gridy.reshape(-1), gridz.reshape(-1)])

# array storing distance and index
nnarraydistance = np.zeros(gridx.reshape(-1).size)
nnarrayindex = np.zeros(gridx.reshape(-1).size)

# NN search
for i in range(gridx.reshape(-1).size): #gridx.reshape(-1).size
  dd, ii = modelKDT.query(np.array([gridx.reshape(-1)[i], gridy.reshape(-1)[i], gridz.reshape(-1)[i]]),k=1)
  nnarraydistance[i] = dd 
  nnarrayindex[i] = ii  
print(nnarraydistance)
print(nnarrayindex)

# Calculate TDF
voxelGridTDF = [1.0 - min( i/(voxelSize*voxelMargin) ,1.0) for i in nnarraydistance];
# print(voxelGridTDF) # long processing time and may make your PC frozen

build kdtree
[4.62346423 4.5601724  4.49821334 ... 4.1806049  4.23638543 4.29377057]
[2693. 2693. 2693. ...  485.  485.  485.]
